# Úkol č. 2 - regrese

## Zdroj dat

Budeme se zabývat predikcí délky dožití v různých zemích a letech.
K dispozici máte trénovací data v souboru `data.csv` a data na vyhodnocení v souboru `evaluation.csv`.

#### Seznam příznaků:

* Year - Rok
* Status - Status rozvinuté nebo rozvojové země
* Life expectancy - Délka dožití v letech - **cílová proměnná, kterou budete predikovat**
* Adult Mortality - Úmrtnost dospělých bez ohledu na pohlaví (pravděpodobnost, že osoby, které dosáhly věku 15 let, zemřou před dosažením věku 60 let (uvedeno na 1 000 osob)).
* infant deaths - počet zemřelých kojenců na 1000 obyvatel
* Alcohol - Alkohol, zaznamenaná spotřeba na obyvatele (15+) (v litrech čistého alkoholu)
* percentage expenditure - Výdaje na zdravotnictví v procentech hrubého domácího produktu na obyvatele (%)
* Hepatitis B - pokrytí očkováním proti hepatitidě B (HepB) u dětí ve věku 1 roku (%)
* Measles - Spalničky - počet hlášených případů na 1000 obyvatel
* BMI - průměrný index tělesné hmotnosti celé populace
* under-five deaths - počet úmrtí dětí do pěti let na 1000 obyvatel
* Polio - proočkovanost proti dětské obrně (Pol3) u dětí ve věku 1 roku (%)
* Total expenditure - Výdaje vládních institucí na zdravotnictví jako procento celkových vládních výdajů (%)
* Diphtheria - pokrytí očkováním proti záškrtu, tetanu a černému kašli (DTP3) u jednoletých dětí (%)
* HIV/AIDS - počet úmrtí na 1 000 živě narozených dětí na HIV/AIDS (0-4 roky)
* GDP - hrubý domácí produkt na obyvatele (v USD)
* Population - počet obyvatel země
* thinness 1-19 years - podíl dětí ve věku 10-19 let s indexem tělesné hmotnosti (BMI) menším než 2 směrodatné odchylky pod mediánem (%)
* thinness 5-9 years - podíl dětí ve věku 5-9 let s indexem tělesné hmotnosti (BMI) menším než 2 směrodatné odchylky pod mediánem (%)
* Income composition of resources - Index lidského rozvoje z hlediska příjmového složení zdrojů (index v rozmezí 0 až 1)
* Schooling - počet let školní docházky (roky)


## Pokyny k vypracování

**Body zadání**, za jejichž (poctivé) vypracování získáte **25 bodů**: 
  * V notebooku načtěte data ze souboru `data.csv`. Vhodným způsobem si je rozdělte na podmnožiny, které Vám poslouží pro trénování (trénovací), porovnávání modelů (validační) a následnou predikci výkonnosti finálního modelu (testovací).
    
  * Proveďte základní předzpracování dat:
    * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném regresním modelu.
    * Nějakým způsobem se vypořádejte s chybějícími hodnotami. _Pozor na metodické chyby!_
    * Můžete využívat i vizualizace. Vše stručně ale náležitě komentujte.
<br /><br />
  * Vytvořte **vlastní implementaci náhodného lesa**. Použijte k tomu níže předpřipravenou kostru.
  
  * Na připravená data aplikujte Vaši předchozí implementaci modelu náhodného lesa. Pro tento model přiměřeně:
    * Okomentujte jeho vhodnost pro daný typ úlohy.
    * Vyberte si hlavní hyperparametry k ladění a najděte jejich nejlepší hodnoty (vzhledem k RMSE).
    * Pro model s nejlepšími hodnotami hyperparametrů určete jeho chybu pomocí RMSE a MAE. _Pozor na metodické chyby!_
    * Získané výsledky vždy řádně okomentujte.
<br /><br />
  * Ze všech zkoušených možností v předchozím kroku vyberte finální model a odhadněte, jakou chybu (RMSE) můžete očekávat na nových datech, která jste doposud neměli k dispozici. _Pozor na metodické chyby!_
    
  * Nakonec načtěte vyhodnocovací data ze souboru `evaluation.csv`. Pomocí finálního modelu napočítejte predikce pro tato data. Vytvořte soubor `results.csv`, ve kterém získané predikce uložíte s využitím tří sloupců: **Country**, **Year** a **Life expectancy**. Tento soubor též odevzdejte (uložte do repozitáře vedle notebooku).

  * Ukázka prvních řádků souboru `results.csv`:
  
```
Country,Year,Life expectancy
Peru,2012,71.4
Peru,2013,72.6
...
```

## Predspracovanie dát

Najskôr si importujem všetky balíčky, čo budem v úlohe potrebovať a nastavím si formát čísel tak, aby sa mi neobjavovali vo vedeckom formáte. Hneď potom si načítam .csv súbor a pozriem sa na základné informácie o ňom (prevažne sa sústredím na chýbajúce hodnoty a typy hodnôt). Na prvý pohľad vidím, že chýbajúcich hodnôt je dosť a že jediný stĺpec, ktorý bude potrebné previesť do číselnej reprezentácie je stĺpec "Status", ktorý bude nadobúdať len dve hodnoty a to 0 a 1 (Developing a Developed).
Na začiatok si ešte dáta prerozdelím na trénovaciu, validačnú a testovaciu množinu.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import root_mean_squared_error,mean_absolute_error

pd.set_option("display.max_columns",None)
np.set_printoptions(precision=5, suppress=True)

data = pd.read_csv("data.csv",index_col=False)
print(data.info())

train_data,test_data = train_test_split(data,test_size=0.5,random_state=42)
val_data,test_data = train_test_split(test_data,test_size=0.5,random_state=42)

data["Country"].unique() #aby som vedel, ktoré všetky štáty sa nachádzajú v datasete pre ich následné rozradenie do kontinentov resp. regiónov

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2718 entries, 0 to 2717
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2718 non-null   object 
 1   Year                             2718 non-null   int64  
 2   Status                           2718 non-null   object 
 3   Life expectancy                  2718 non-null   float64
 4   Adult Mortality                  2718 non-null   float64
 5   infant deaths                    2718 non-null   int64  
 6   Alcohol                          2564 non-null   float64
 7   percentage expenditure           2718 non-null   float64
 8   Hepatitis B                      2188 non-null   float64
 9   Measles                          2718 non-null   int64  
 10  BMI                              2692 non-null   float64
 11  under-five deaths                2718 non-null   int64  
 12  Polio               

array(['Afghanistan', 'Albania', 'Algeria', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia (Plurinational State of)', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Burundi', "Côte d'Ivoire", 'Cabo Verde',
       'Cambodia', 'Cameroon', 'Canada', 'Central African Republic',
       'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czechia',
       "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Fiji',
       'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany',
       'Ghana', 'Greece', 'Grenad

Najskôr teda rozkategorizujem krajiny podľa odpovedajúcich svetadielov resp. regiónov a každý svetadiel (región) uložím ako zvlášť stĺpec s odpovedajúcim názvom. Takže novo-vzniknuté stĺpce budú "North_America", "Latin_America", "Caribic" "Africa", "Western_Europe", "Eastern_Europe_and_Balkan","Poor_Asia", "Rich_Asia", "Australia_oceania".
Aj napriek jasným geografickým polohám som pri niektorom zadeľovaní krajín do regiónov prihliadol aj na demografické a socioekonomické parametry.
* Cyprus aj napriek geografickej lokácii som zaradil medzi západoeurópske krajiny kvôli jeho členstvu v EÚ.
* Izrael, aj napriek jeho geografickej polohe som zaradil medzi krajiny západnej Európy, pretože v ňom žije prevažná časť európanov a aj ich štátny systém sa podobá európskemu.
* Japonsko, Singapur, Južnú Kóreu a bohaté arabské krajiny som zaradil do bohatej Ázie, pretože tieto sa výrazne odlišujú od iných ázijských štátov. 
* Čínu som zaradil takisto medzi krajiny bohatej Ázie. Aj keď väčšina rozlohy Číny sú hory a púšť, jej obyvateľstvo je prevažne sústredené na jej východ, ktoré je veľkým obchodným centrom a teda oveľa viac rozvinuté.
* Rusko som zaradil do východnej Európy pretože väčšina obyvateľstva sa nachádza v európskej časti Ruska.
* Zvyšné krajiny boli zaradené podľa odpovedajúcej geografickej polohy.

Vytvorím si funkciu new_continent, ktorá pre každý dataset vytvorí stĺpce podľa kontinentov. Vo funkcii rovnako nahradím všetky hodnoty v stĺpci "Status" číselnými hodnotami. Hodnota "Developing" bude nahradená nulou a hodnota "Developed" bude nahradená jednotkou. 

In [2]:
#roztriedenie štátov do regiónov – manuálne

north_america = ["United States of America","Canada"]
latin_america = ["Argentina","Belize","Bolivia (Plurinational State of)","Brazil","Chile","Colombia","Costa Rica","Ecuador","El Salvador","Guatemala",
                 "Guyana","Honduras","Mexico","Nicaragua","Panama","Paraguay","Peru","Uruguay","Venezuela (Bolivarian Republic of)"]
caribic = ["Antigua and Barbuda","Bahamas","Barbados","Cuba","Dominican Republic","Grenada","Haiti","Jamaica","Saint Lucia",
           "Saint Vincent and the Grenadines","Suriname","Trinidad and Tobago"]
africa = ["Algeria","Angola","Benin","Botswana","Burkina Faso","Burundi","Côte d'Ivoire","Cabo Verde","Cameroon","Central African Republic","Chad","Comoros",
          "Congo","Democratic Republic of the Congo","Djibouti","Egypt","Equatorial Guinea","Eritrea","Ethiopia","Gabon","Gambia","Ghana","Guinea",
          "Guinea-Bissau","Kenya","Lesotho","Liberia","Libya","Madagascar","Malawi","Mali","Mauritania","Mauritius","Morocco","Mozambique","Namibia",
          "Niger","Nigeria","Rwanda","Sao Tome and Principe","Senegal","Seychelles","Sierra Leone","Somalia","South Africa","South Sudan","Sudan",
          "Swaziland","Togo","Tunisia","Uganda","United Republic of Tanzania","Zambia","Zimbabwe"]
western_europe = ["Austria","Belgium","Cyprus","Czechia","Denmark","Estonia","Finland","France","Germany","Greece","Hungary","Iceland","Ireland","Israel",
               "Italy","Latvia","Lithuania","Luxembourg","Malta","Netherlands","Norway","Poland","Portugal","Slovakia","Slovenia","Spain","Sweden",
               "Switzerland","United Kingdom of Great Britain and Northern Ireland"]
eastern_europe_and_balkan = ["Albania","Belarus","Bosnia and Herzegovina","Bulgaria","Croatia","Montenegro","Republic of Moldova","Romania","Russian Federation",
               "Serbia","The former Yugoslav republic of Macedonia","Ukraine"]
poor_asia = ["Afghanistan","Armenia","Azerbaijan","Bahrain","Bangladesh","Bhutan","Brunei Darussalam","Cambodia","Democratic People's Republic of Korea",
        "Georgia","India","Indonesia","Iran (Islamic Republic of)","Iraq","Jordan","Kazakhstan","Kyrgyzstan","Lao People's Democratic Republic","Lebanon",
        "Malaysia","Maldives","Mongolia","Myanmar","Nepal","Oman","Pakistan","Papua New Guinea","Philippines","Saudi Arabia","Sri Lanka",
        "Syrian Arab Republic","Tajikistan","Thailand","Timor-Leste","Turkey","Turkmenistan","Uzbekistan","Viet Nam","Yemen"]
rich_asia = ["Japan","China","Kuwait","Qatar","Republic of Korea","Singapore","United Arab Emirates"]
australia_oceania = ["Australia","Fiji","Kiribati","Micronesia (Federated States of)","New Zealand","Samoa","Solomon Islands","Tonga","Vanuatu"]

#definovanie dvoch pomocných radov, ktoré využijem v nižšie definovanej funkcii

region_arrays = [north_america,latin_america,caribic,africa,western_europe,eastern_europe_and_balkan,poor_asia,rich_asia,australia_oceania]
regions = ["North_America","Latin_America","Caribic","Africa","Western_Europe","Eastern_Europe_and_Balkan", "Poor_Asia","Rich_Asia","Australia_Oceania"]

#jedna funkcia pre všetky tri množiny dát

def new_continent(countries,new_continent,dataframe):
    dataframe["Status"] = dataframe["Status"].replace("Developing",0)
    dataframe["Status"] = dataframe["Status"].replace("Developed",1)
    dataframe[new_continent] = dataframe["Country"] #vytvorenie nového príznaku kontinentu, do ktorého sa nakopírujú hodnoty zo stĺpca "Country"
    for country in dataframe["Country"]: #iterácia cez stĺpec "Country"
        if country in countries: #Ak je aktuálna hodnota stĺpca "Country" v konkrétnom regióne, tak sa nahradí 1, inak 0
            dataframe[new_continent] = dataframe[new_continent].replace(country,1)
        else:
            dataframe[new_continent] = dataframe[new_continent].replace(country,0)

#vytvorenie nových príznakov podľa regiónov

for region,region_name in zip(region_arrays,regions):
    new_continent(region,region_name,train_data)

for region,region_name in zip(region_arrays,regions):
    new_continent(region,region_name,test_data)

for region,region_name in zip(region_arrays,regions):
    new_continent(region,region_name,val_data)


Aj napriek tomu, že príznakov v datasete je množstvo, aj po dôkladnom preskúmaní sa z nich nič moc vydolovať nedá. Väčšinou je známe len relatívne zastúpenie nejakého javu pre určitú vekovú skupinu, o ktorej nemám žiadne informácie. Napríklad mám k dispozícií príznaky znázorňujúce relatívny počet zaočkovaných detí v určitom veku, no nepoznám celkový počet detí v danej krajine a ani to, či dané dieťa má viac vakcín, alebo len jednu. Preto ďalšie prípadné sčítavanie, prípadne kombinovanie stĺpcov už robiť nebudem.

Teraz sa vysporiadam s chýbajúcimi hodnotami v datasete. Chýbajúce hodnoty sa nachádzajú v stĺpcoch "Alcohol", "Hepatitis B", "BMI", "Polio", "Total expenditure", "Diphtheria", "GDP", "Population", "thinness 1-19 years", "thinness 5-9 years", "Income composition of resources" a "Schooling". So všetkými chýbajúcimi hodnotami sa vysporiadam tak, že z trénovacieho datasetu vyberiem dané príznaky pre jednotlivé svetadiely resp. regióny, spriemerujem ich a tieto priemery dosadím za chýbajúce hodnoty aj na validačnej aj testovacej množine. Je totiž jasné, že napríklad spotreba alkoholu bude vyššia vo východnej Európe a na Balkáne, než v arabských krajinách. Preto pracujem s priemernými hodnotami pre každý región zvlášť.

In [3]:
missing_values_columns = ["Alcohol","Hepatitis B", "BMI","Polio","Total expenditure","Diphtheria","GDP","Population","thinness  1-19 years",
                          "thinness 5-9 years","Income composition of resources","Schooling"]
query_regions = ["North_America==1","Latin_America==1","Caribic==1","Africa==1","Western_Europe==1","Eastern_Europe_and_Balkan==1", "Poor_Asia==1",
           "Rich_Asia==1","Australia_Oceania==1"] #V rade musí byť rovnako zapísaná podmienka ako string, inak to metóda query nevie vyhodnotiť

means_dict = {} #sem budem ukladať jednotlivé priemer stĺpcovy pre odpovedajúci región. Stĺpcov je 12, toľko bude aj key-value pairs v dict.
                # Každý key bude mať k sebe value typu array, ktorý má dĺžku 9 (zvlášť priemer pre konkrétny región)

#spriemerovanie stĺpcov, v ktorých sú chýbajúce hodnoty pre trénovací dataset. Pre každý región zvlášť.

for column in missing_values_columns:
    means_arr = []
    for query in query_regions:
        result_mean = train_data.query(query)[column].mean()
        means_arr.append(result_mean)
    means_dict[column] = means_arr
    

#funkcia, pomocou ktorej dosadím priemerné hodnoty miesto prázdnych. Pre každý región zvlášť.

def fillNAN(data,column):
    filled_nan = np.array([])
    for query,mean in zip(query_regions,means_dict[column]):
        result = data.query(query)[column]
        result_filled = result.replace(np.nan,mean)
        filled_nan = np.append(filled_nan,result_filled)
    data[column] = data[column].replace(np.array(data[column]),filled_nan)

#dosadenie priemerov trénovacej množiny do jednotlivých množín

for column in missing_values_columns:
    fillNAN(train_data,column)

for column in missing_values_columns:
    fillNAN(val_data,column)

for column in missing_values_columns:
    fillNAN(test_data,column)


Ďalší krok predspracovania dát bude vyhodenie stĺpca "Country" zo všetkých troch množín.

In [4]:
train_data = train_data.drop(["Country"],axis=1)
val_data = val_data.drop(["Country"],axis=1)
test_data = test_data.drop(["Country"],axis=1)

Ako posledný krok predspracovania dát bude vykreslenie korelačnej matice príznakov z trénovacej množiny. Korelačná matica mi povie, či v matici príznakov mám také, ktoré sú na sebe nejakým spôsobom závislé. Síce korelačnú maticu vykresľujem len pre trénovaciu množinu, no pre validačnú a testovaciu budú korelácie podobné, pretože tieto množiny boli zhotovené náhodným výberom, a teda sú reprezentatívne.

Pri tvorbe modelu lineárnej regresie ma zaujíma ale len špeciálny typ závislosti, a to je lineárna závislosť. Či sú príznaky na sebe lineárne závislé, to už mi korelačná matica nepovie. Prípadnú lineárnu závislosť príznakov overím dosadením predpovedaných koeficientov w_hat do normálnej rovnice. 

In [5]:
Xtrain = train_data.drop("Life expectancy",axis=1)
Ytrain = train_data["Life expectancy"]

Xval = val_data.drop("Life expectancy",axis=1)
Yval = val_data["Life expectancy"]

Xtest = test_data.drop("Life expectancy",axis=1)
Ytest = test_data["Life expectancy"]

## Random Forest

Model náhodného lesa pozostáva z viacerých modelov rozhodovacích stromov, z ktorých každý jeden má rovnaké hyperparametre ale jeho natrénovanie prebehlo na rozdielnych dátach vďaka metóde bootstrapu. Tieto jednotlivé stromy robia svoje vlastné predikcie a predikcia náhodného lesa je vznikne spriemerovaním týchto hodnôt v prípade regresnej úlohy. V prípade klasifikačnej úlohy sa vyberie najčastejšie sa opakujúca hodnota. Stromy používané v tomto modeli sú často plytšie a nepredikujú až tak dobre, no v konečnom dôsledku predikcia samotného náhodného lesa je kvalitná. Model je rovnako robustný a imúnny voči preučeniu. Pri veľkom množstve zadaných stromov sa stromy medzi sebou začnú opakovať ale neprispôsobia sa trénovacím dátam. 

Začnem vlastnou implementáciou modelu náhodného lesa.

In [6]:
tree_estimators = {}

class CustomRandomForest:
    

    def __init__(self, n_estimators, max_samples_absolute, max_depth, criterion):
        self.n_estimators = n_estimators
        self.max_samples_absolute = max_samples_absolute
        self.max_depth = max_depth
        self.criterion = criterion

        
    def fit(self, Xtrain, Ytrain):
        for estimator in range(1,self.n_estimators+1):
            #aby Randomstate bol vždy rovnaký pri bootstrape z Xtrain aj Ytrain, aby si jednotlivé hodnoty príznakov a predikcí odpovedali
            RS = estimator 
            bootstrap_X = Xtrain.sample(n=self.max_samples_absolute,replace=True,axis=0,random_state=RS)
            bootstrap_Y = Ytrain.sample(n=self.max_samples_absolute,replace=True,axis=0,random_state=RS)
            tree = DecisionTreeRegressor(
                max_depth=self.max_depth,criterion=self.criterion,
                ).fit(bootstrap_X,bootstrap_Y)
            #uloženie natrénovaných stromov do slovníka
            tree_estimators.update({estimator:tree})

        
    def predict(self, X):
        #zadefinovanie prázdneho slovníka a arrays, do ktorých budem v cykloch ukladať výsledky
        prediction = {} #slovník, kde sa uložia predpovede jednotlivých stromov vo formáte {number:array}.
        prediction_per_country = [] #tu sa uložia predpovede dĺžky života stromov vždy len pre jednu krajinu.
        Ypredicted = [] #tu sa vloží priemer predpovedí dĺžky života pre danú krajinu z arrayu prediction_per_country.

        for estimator in range(1,self.n_estimators+1):
            tree = tree_estimators[estimator]
            prediction.update({estimator:tree.predict(X)})

        for index in range(len(X)):
            for estimator in range(1,self.n_estimators+1):
                prediction_per_country.append(prediction[estimator][index])
            result = np.mean(prediction_per_country)
            Ypredicted.append(result)
            prediction_per_country = []

        return Ypredicted


Po úspešnej ručnej implementácii modelu náhodného lesa som sa rozhodol ladiť hyperparametre. No ladenie muselo prebiehať zvlášť, pretože v prípade, kedy som sa snažil ladiť všetky hyperparametre naraz mi vyšlo, že model počíta približne 20,000,000 scenárov, čo ani po 6 hodinách spusteného kódu nebolo spočítané. Preto som sa rozhodol ladiť zvlášť hyperparametre criterion, max_depth, n_estimators a max_samples_absolute. Po 37 minútach mi kód vypľul výsledok, kedy najvhodnejšia kombinácia parametrov bola max_depth = 13, criterion = poisson, n_estimators = 16 a max_samples_absolute = 685 s validačnou RMSE = 2.699. Tieto hodnoty sa môžu vďaka náhode pri rôznych dátach trochu líšiť, no nebude to o moc.

Výpočet v nasledovnom bloku kódu trval dlho, preto bunka nie je spustená.

In [ ]:
param_grid = {
    'criterion': ['squared_error','absolute_error', 'poisson', 'friedman_mse'],
    'max_depth': range(2,15),
    'n_estimators': range(10,50),
    'max_samples_absolute': range(200,1000),
}

param_comb = ParameterGrid(param_grid)

RF_val_RMSE = []
for params in param_comb:
    RF_model = CustomRandomForest(n_estimators=params['n_estimators'],max_depth=params['max_depth'],criterion=params['criterion'],max_samples_absolute=params['max_samples_absolute'])
    RF_model.fit(Xtrain, Ytrain)
    RF_val_RMSE.append(root_mean_squared_error(Yval, RF_model.predict(Xval)))

    print("Val RMSE je:", RF_val_RMSE[-1],
          "Hĺbka je:", params['max_depth'],
          "Kritérium:",params['criterion'],
          "Počet stromov:",params['n_estimators'],
          "Počet max samples:",params['max_samples_absolute'])

best_params_min = param_comb[np.argmin(RF_val_RMSE)]
best_params_min

Teraz si ešte vyladené hyperparametre náhodného lesa uložím do zvlášť premennej a vypočítam validačnú RMSE pre finálny model náhodného lesa.

In [8]:
hp_RF = {
         'max_samples_absolute': 685,
         'max_depth': 13,
         'n_estimators': 16,
         'criterion': 'poisson'
         }

RF_model_final = CustomRandomForest(hp_RF['n_estimators'],hp_RF['max_samples_absolute'],hp_RF['max_depth'],hp_RF['criterion'])
RF_model_final.fit(Xtrain,Ytrain)

print("Validačná RMSE pre najlepší model náhodného lesa je:",root_mean_squared_error(Yval,RF_model_final.predict(Xval)))
print("Validačná MAE pre najlepší model náhodného lesa je:",mean_absolute_error(Yval,RF_model_final.predict(Xval)))

Validačná RMSE pre najlepší model náhodného lesa je: 2.7053740432209374
Validačná MAE pre najlepší model náhodného lesa je: 1.907678512503956


Model náhodného lesa použijem na predikciu RMSE a MAE na dátach, ktoré som doposiaľ k dispozícií nemal, teda na testovacej množine. Ako vidíme, výsledok je veľmi podobný očakávanej hodnote RMSE a MAE získanej na validačných dátach.

In [9]:
print("Testovacia RMSE pre finálny model náhodného lesa je:",root_mean_squared_error(Ytest,RF_model_final.predict(Xtest)))
print("Testovacia MAE pre finálny model náhodného lesa je:",mean_absolute_error(Ytest,RF_model_final.predict(Xtest)))

Testovacia RMSE pre finálny model náhodného lesa je: 2.732917398879111
Testovacia MAE pre finálny model náhodného lesa je: 1.9297341958398118


Posledný krok je už len načítanie dát zo súboru evaluation.csv a ich transformácia. Vyčlenenie stĺpca "Country" a predikcia strednej dĺžky života na krajinu.

In [10]:
final_data = pd.read_csv("evaluation.csv")

# Po načítaní si upravím final_data najskôr vytvorením stĺpcov regiónov a potom doplnením chýbajúcich hodnôt podľa priemerov vypočítaných pre
# train_data. 

for region,region_name in zip(region_arrays,regions):
    new_continent(region,region_name,final_data)

for column in missing_values_columns:
    fillNAN(final_data,column)

# A ešte vyhodím z dát stĺpec "Country" a uložím ho do premennej.

final_data_country = final_data["Country"]
final_data_year = final_data["Year"]
final_data = final_data.drop("Country",axis=1)

Ypredicted_final = RF_model_final.predict(final_data)



Na koniec už len posledné úpravy, aby všetky datatypes boli kompatibilné, konverzia radov na dataframe a export dát do .csv súboru.

In [11]:
Ypredicted_final_array = np.array(Ypredicted_final)

result = pd.DataFrame([final_data_country,final_data_year,Ypredicted_final_array]).transpose()
result = result.rename(columns={"Country":"Country","Year":"Year","Unnamed 0":"Life expectancy"})

#result.to_csv("results.csv",index=False) #odkomentované aby sa export neopakoval